In [ ]:
from queue import Queue
from threading import Thread
import sys
import time
import cv2

class VideoReader:
    def __init__(self, video_file, queue_size=128):
        # initialize the file video stream along with the boolean
        # used to indicate if the thread should be stopped or not
        self.stream = cv2.VideoCapture(video_file)
        self.stopped = False
        # initialize the queue used to store frames read from
        # the video file
        self.Q = Queue(maxsize=queue_size)
        self.frame_num = 0
        self.false_count = 0 
        print('reading video from ', video_file)
        


    def start(self):
        # start a thread to read frames from the file video stream
        t = Thread(target=self.update, args=())
        t.daemon = True
        t.start()
        time.sleep(5)
        print('starting video reader...')
        return self

    def update(self):
        # keep looping infinitely
#         frame_num = 0
        print('update starting...')
        while True:
#             if frame_num == 100:
#                 break
            # if the thread indicator variable is set, stop the
            # thread
            if self.stopped:
                return

            # otherwise, ensure the queue has room in it
            if not self.Q.full():
                # read the next frame from the file

                (grabbed, frame) = self.stream.read()
                self.frame_num += 1
                
                
                # add the frame to the queue
                if not grabbed:
                    if self.false_count > 500:
                        print('no more frames')
                        self.stop()
                        return
                    else:
                        if self.frame_num % 25 == 0:
                            print('queue, {}, {}'.format(self.frame_num, grabbed))
                        self.false_count += 1
                else:
                    self.Q.put(frame)
                    self.false_count = 0
                

    def read(self):
        # return next frame in the queue
        return self.Q.get()


    def more(self):
        # return True if there are still frames in the queue
        return self.Q.qsize() > 0

    def stop(self):
        # indicate that the thread should be stopped
        self.stopped = True
        self.stream.release()
        print('stopped')
    
    def is_stopped(self):
        return self.stopped
    
    def is_full(self):
        return self.Q.full()
        
        
    def close(self):
        self.stop()

In [ ]:
import glob
parent_video_folder = '.../kasanka-bats/gopros' 

video_files = []
for date in ['17Nov']:
#     video_files.extend(glob.glob(parent_video_folder + date + '/*/*.MP4'))
    video_files.extend(glob.glob(parent_video_folder + date + '/card-f/*.MP4'))
video_files.sort()
    # vf = glob.glob(parent_video_folder + '/14_11_g/*.MP4')
    # vf.sort()
    # video_files.extend(vf)
print(*video_files, sep='\n')

In [ ]:
import glob
import os
from multiprocessing import Pool
skip_existing_folders = False
save_frames = True
process_every_n_frames = 1



# existing_image_folders = glob.glob(output_folder + '/*')
# existing_folder_names = [os.path.basename(image_folder) for image_folder in existing_image_folders]
        


    
def pull_out_frames(video_file):
    
    parent_output_folder = '.../kasanka-bats/frames/'
    parent_output_folder = '.../Elements/bats'

    output_folder = os.path.join(parent_output_folder, video_file.split('/')[-3], video_file.split('/')[-2])
    
    observation_name = video_file.split('/')[-2]

    video_name = os.path.basename(video_file)
    video_name = video_name.split('.')[-2]

    if skip_existing_folders:
        if video_name in existing_folder_names:
            return

    print('Processing', video_name)

    if save_frames:
        
        output_folder = os.path.join(output_folder, video_name)

        if not os.path.exists(output_folder):
            os.makedirs(output_folder)

        print('saving images to ', output_folder)
    else:
        print('frames are not being saved')





    frame_count = 0
    video_stream = VideoReader(video_file).start()
    time.sleep(1)
    last_frame = None
    


    while video_stream.more():
        
        


        if video_stream.is_full() or video_stream.is_stopped():

            if frame_count % 4000 == 0:
                print(frame_count, 'frames processed')

            frame = video_stream.read()

            if save_frames:
                if frame_count % process_every_n_frames == 0:
                    frame_name = "{}_{:05d}".format(os.path.splitext(video_name)[0], frame_count)
                    frame_file = os.path.join(output_folder, frame_name + '.jpg')
                    cv2.imwrite(frame_file, frame)
                    


            frame_count += 1

    video_stream.stop()
    

if __name__ == '__main__':


    with Pool(processes=4) as pool:
        pool.map(pull_out_frames, video_files[2:])
#         connected_distances, connected_sizes = find_tracks_in_clips(file_dicts[0])


In [ ]:
frame_file

In [ ]:
video_files

In [ ]:
import glob 
import cv2

parent_video_folder = '/Volumes/TTT_Paris/bats/video/14_11/14_11_d'

video_files = glob.glob(parent_video_folder + '/*.MP4')

stream = cv2.VideoCapture(video_files[0])

for i in range(300):
    (grabbed, frame) = stream.read()
    print(i, grabbed)